In [1]:
import string
import random

In [61]:
from datetime import datetime, timedelta
import numpy as np

src_accounts = [user_ids[abs(int(np.random.normal(5000, 2500)))% 10000] for _ in range(100000)]
amounts = [ round(np.random.uniform (100, 10000), 2) for _ in range(100000)]
dest_accounts = [''.join(random.choices(string.digits + string.ascii_lowercase, k=6)) for _ in range(100000)]
is_frauds = [np.random.binomial(1, 0.05) for _ in range(100000)]

timestamps = []
ts = datetime.now() - timedelta(days=14)
for i in range(100000):
    timestamps.append(ts)
    ts += timedelta(seconds=np.random.uniform(0, 24))

In [67]:
transactions = pd.DataFrame.from_dict({
    "src_account": src_accounts,
    "amount": amounts,
    "dest_account": dest_accounts,
    "is_fraud": is_frauds,
    "timestamp": timestamps
})

transactions = transactions.sort_values(by='timestamp', ascending=False)

transactions

,src_account,amount,dest_account,is_fraud,timestamp
0,7859ge,7078.83,waqmx5,0,2021-05-21 08:10:12.039737
1,yzziue,7851.83,tgx086,0,2021-05-21 08:10:24.464622
2,bgf8nl,6016.44,q0ltxc,0,2021-05-21 08:10:36.208894
3,jiaxoq,2573.43,ct01il,0,2021-05-21 08:10:54.177049
4,u49qmt,6743.81,397mqf,0,2021-05-21 08:11:16.870868
...,...,...,...,...,...
99995,xqjsd3,6856.42,2z3w39,0,2021-06-04 05:21:23.834089
99996,1tyh8p,8527.89,h5jgwy,0,2021-06-04 05:21:37.946295
99997,mfj3xt,4651.57,xvjv67,0,2021-06-04 05:21:39.853131
99998,l0e31n,9771.14,savnzy,0,2021-06-04 05:22:03.112553


In [138]:
# For some reason, this loads into BQ sorted by src_account
# Needed to sort in BQ after this
transactions.to_gbq(destination_table="fraud_tutorial.transactions", project_id="feast-oss")

1it [00:18, 18.60s/it]


In [4]:
import pandas as pd

sql = """
    SELECT *
    FROM `feast-oss.fraud_tutorial.transactions`
"""

# Run a Standard SQL query using the environment's default project
transactions = pd.read_gbq(sql, dialect='standard')

Downloading: 100%|██████████| 100000/100000 [00:04<00:00, 23842.20rows/s]


In [6]:
transactions

,src_account,amount,dest_account,is_fraud,timestamp
0,782k16,9749.99,24bhqv,0,2021-06-04 05:22:08.142090+00:00
1,l0e31n,9771.14,savnzy,0,2021-06-04 05:22:03.112553+00:00
2,mfj3xt,4651.57,xvjv67,0,2021-06-04 05:21:39.853131+00:00
3,1tyh8p,8527.89,h5jgwy,0,2021-06-04 05:21:37.946295+00:00
4,xqjsd3,6856.42,2z3w39,0,2021-06-04 05:21:23.834089+00:00
...,...,...,...,...,...
99995,u49qmt,6743.81,397mqf,0,2021-05-21 08:11:16.870868+00:00
99996,jiaxoq,2573.43,ct01il,0,2021-05-21 08:10:54.177049+00:00
99997,bgf8nl,6016.44,q0ltxc,0,2021-05-21 08:10:36.208894+00:00
99998,yzziue,7851.83,tgx086,0,2021-05-21 08:10:24.464622+00:00


In [10]:
from datetime import datetime, timedelta
import pytz

t = transactions

aggregation_end = datetime.now(tz=pytz.UTC)

# last week's transactions
lwt = t[(t['timestamp'] < aggregation_end ) & (t['timestamp'] >= aggregation_end - timedelta(days=7))]

# users with fraudulent transactions
fu = lwt[lwt['is_fraud'] == 1]\
        .drop_duplicates(subset='src_account')[['src_account', 'is_fraud']]\
        .rename(columns={'is_fraud': 'user_has_fraudulent_transactions_7d'})        


In [11]:
fu

,src_account,user_has_fraudulent_transactions_7d,feature_timestamp
27,zqvbs4,1,2021-06-07 12:45:10.214026
37,a9l0te,1,2021-06-07 12:45:10.214026
62,z2lnqe,1,2021-06-07 12:45:10.214026
82,xv1ul5,1,2021-06-07 12:45:10.214026
112,6ua5v6,1,2021-06-07 12:45:10.214026
...,...,...,...
25188,kr123d,1,2021-06-07 12:45:10.214026
25243,y7dobz,1,2021-06-07 12:45:10.214026
25303,wija9d,1,2021-06-07 12:45:10.214026
25375,u269is,1,2021-06-07 12:45:10.214026


In [23]:
# feature table
ff = pd.merge(t, fu, on='src_account',how='left')\
        .fillna(0)\
        [['src_account', 'user_has_fraudulent_transactions_7d']].rename(columns={"src_account": "user_id"})

ff['feature_timestamp'] = datetime.now()

ff

,user_id,user_has_fraudulent_transactions_7d,feature_timestamp
0,782k16,0.0,2021-06-07 12:58:28.318652
1,l0e31n,0.0,2021-06-07 12:58:28.318652
2,mfj3xt,1.0,2021-06-07 12:58:28.318652
3,1tyh8p,0.0,2021-06-07 12:58:28.318652
4,xqjsd3,0.0,2021-06-07 12:58:28.318652
...,...,...,...
99995,u49qmt,0.0,2021-06-07 12:58:28.318652
99996,jiaxoq,0.0,2021-06-07 12:58:28.318652
99997,bgf8nl,0.0,2021-06-07 12:58:28.318652
99998,yzziue,0.0,2021-06-07 12:58:28.318652


In [20]:
from datetime import datetime, timedelta
import pytz

sql = """
SELECT *
FROM `feast-oss.fraud_tutorial.transactions`
"""

t = pd.read_gbq(sql, dialect='standard')

def generate_fraud_features(aggregation_end):

    user_ids = t['src_account'].drop_duplicates()

    # last week's transactions
    lwt = t[(t['timestamp'] < aggregation_end ) & (t['timestamp'] >= aggregation_end - timedelta(days=7))]

    # users with fraudulent transactions
    fu = lwt[lwt['is_fraud'] == 1]\
            .drop_duplicates(subset='src_account')[['src_account', 'is_fraud']]\
            .rename(columns={'is_fraud': 'user_has_fraudulent_transactions_7d'}) \
    # feature table
    ff = pd.merge(user_ids, fu, on='src_account',how='left')\
            .fillna(0)\
            [['src_account', 'user_has_fraudulent_transactions_7d']].rename(columns={"src_account": "user_id"})

    ff['feature_timestamp'] = aggregation_end
    ff.to_gbq(destination_table="fraud_tutorial.user_has_fraudulent_transactions", project_id="feast-oss", if_exists='append')

def backfill_features(start_date, interval, iters):
    aggregation_end_date = start_date
    for _ in range(iters):
        print(f"generating features as of {aggregation_end_date}")
        generate_fraud_features(aggregation_end=aggregation_end_date)
        aggregation_end_date += interval
    
ff = backfill_features(
    start_date=datetime.now(tz=pytz.UTC) - timedelta(days=7), 
    interval=timedelta(days=1), 
    iters=7
)


Downloading: 100%|██████████| 100000/100000 [00:03<00:00, 25626.05rows/s]
generating features as of 2021-06-03 16:42:14.250521+00:00

1it [00:03,  3.78s/it]
generating features as of 2021-06-04 16:42:14.250521+00:00

1it [00:02,  2.77s/it]
generating features as of 2021-06-05 16:42:14.250521+00:00

1it [00:02,  2.72s/it]
generating features as of 2021-06-06 16:42:14.250521+00:00

1it [00:05,  5.13s/it]
generating features as of 2021-06-07 16:42:14.250521+00:00

1it [00:02,  2.27s/it]
generating features as of 2021-06-08 16:42:14.250521+00:00

1it [00:02,  2.61s/it]

0it [00:00, ?it/s]generating features as of 2021-06-09 16:42:14.250521+00:00
1it [00:05,  5.17s/it]


In [24]:
ff.to_gbq(destination_table="fraud_tutorial.user_has_fraudulent_transactions", project_id="feast-oss", if_exists='replace')

1it [00:06,  6.93s/it]


In [25]:
import numpy as np

user_ids = t['src_account'].drop_duplicates()

user_features = pd.DataFrame.from_dict({
    "user_id": user_ids,
    "credit_score": [int(np.random.uniform(350, 850)) % 850 for _ in range(9944)],
    "account_age_days": [int(np.random.uniform(25, 960)) for _ in range(9944)],
    "user_has_2fa_installed": [int(np.random.uniform(0, 2)) for _ in range(9944)],
    "feature_timestamp": [datetime.now() for _ in range(9944)]
})

user_features

,user_id,credit_score,account_age_days,user_has_2fa_installed,feature_timestamp
0,782k16,626,799,1,2021-06-07 12:59:14.813413
1,l0e31n,648,889,1,2021-06-07 12:59:14.813418
2,mfj3xt,603,383,1,2021-06-07 12:59:14.813419
3,1tyh8p,808,701,0,2021-06-07 12:59:14.813419
4,xqjsd3,351,428,0,2021-06-07 12:59:14.813420
...,...,...,...,...,...
97279,h1p7lk,518,407,1,2021-06-07 12:59:14.818469
97325,n120dt,595,927,1,2021-06-07 12:59:14.818470
97818,txk4ui,583,872,1,2021-06-07 12:59:14.818470
98870,j72zdi,685,114,0,2021-06-07 12:59:14.818471


In [26]:
user_features.to_gbq(destination_table="fraud_tutorial.user_account_features", project_id="feast-oss", if_exists='replace')

1it [00:05,  5.71s/it]


In [125]:
x = np.rando2021-06-04 12:56:23.851782 UTC
m.uniform(0, 2)
print(x)
int(x)

1.7273118602433915


1

In [2]:
import pandas as pd

sql = """
    SELECT *
    FROM `feast-oss.fraud_tutorial.transactions`
"""

# Run a Standard SQL query using the environment's default project
transactions = pd.read_gbq(sql, dialect='standard')

latest_time = transactions['timestamp'].max()

Downloading: 100%|██████████| 100000/100000 [00:04<00:00, 20677.91rows/s]


In [7]:
from datetime import datetime
datediff = datetime.now() - latest_time.replace(tzinfo=None)

transactions['timestamp'] = transactions['timestamp'] + datediff

transactions

,src_account,amount,dest_account,is_fraud,timestamp
0,782k16,9749.99,24bhqv,0,2021-06-10 11:53:31.513514+00:00
1,l0e31n,9771.14,savnzy,0,2021-06-10 11:53:26.483977+00:00
2,mfj3xt,4651.57,xvjv67,0,2021-06-10 11:53:03.224555+00:00
3,1tyh8p,8527.89,h5jgwy,0,2021-06-10 11:53:01.317719+00:00
4,xqjsd3,6856.42,2z3w39,0,2021-06-10 11:52:47.205513+00:00
...,...,...,...,...,...
99995,u49qmt,6743.81,397mqf,0,2021-05-27 14:42:40.242292+00:00
99996,jiaxoq,2573.43,ct01il,0,2021-05-27 14:42:17.548473+00:00
99997,bgf8nl,6016.44,q0ltxc,0,2021-05-27 14:41:59.580318+00:00
99998,yzziue,7851.83,tgx086,0,2021-05-27 14:41:47.836046+00:00


In [9]:
transactions.to_gbq(destination_table="fraud_tutorial.transactions", project_id="feast-oss", if_exists='replace')

1it [00:09,  9.47s/it]


In [5]:
import pandas as pd
from datetime import datetime, timedelta

sql = """
    SELECT *
    FROM `feast-oss.fraud_tutorial.user_account_features`
"""

user_features = pd.read_gbq(sql, dialect='standard')

user_features['feature_timestamp'] = datetime.now() - timedelta(days=7)

user_features

Downloading: 100%|██████████| 9944/9944 [00:00<00:00, 12317.33rows/s]


,user_id,credit_score,account_age_days,user_has_2fa_installed,feature_timestamp
0,41sozr,512,700,0,2021-06-03 12:11:13.032174
1,h8nr8u,512,157,0,2021-06-03 12:11:13.032174
2,shid6v,512,509,0,2021-06-03 12:11:13.032174
3,rbcoqw,512,742,0,2021-06-03 12:11:13.032174
4,hew545,512,327,0,2021-06-03 12:11:13.032174
...,...,...,...,...,...
9939,nsgtkp,767,891,1,2021-06-03 12:11:13.032174
9940,4dlidj,767,855,1,2021-06-03 12:11:13.032174
9941,1z87hk,767,271,1,2021-06-03 12:11:13.032174
9942,ffqerm,767,829,1,2021-06-03 12:11:13.032174


In [6]:
user_features.to_gbq(destination_table="fraud_tutorial.user_account_features", project_id="feast-oss", if_exists='replace')

1it [00:04,  4.62s/it]
